## LlamaIndex + DeepEval Integration

This code tutorial shows how you can easily integrate LlamaIndex with DeepEval. DeepEval makes it easy to unit-test your LLMs.

You can read more about the DeepEval framework here: https://docs.confident-ai.com/docs/framework

Feel free to check out our repository here: https://github.com/confident-ai/deepeval

![Framework](https://docs.confident-ai.com/assets/images/llm-evaluation-framework-example-b02144720026b6d49b1e04d8a99d3d33.png)

## Set-up and Installation

We recommend setting up and installing via pip!

In [ ]:
!pip install -q -q llama-index
!pip install -U -q deepeval

This step is optional and only if you want a server-hosted dashboard! (Psst I think you should!)

In [ ]:
!deepeval login

## Testing for factual consistency

In [ ]:
from llama_index.response.schema import Response
from typing import List
from llama_index.schema import Document
from deepeval.metrics.factual_consistency import FactualConsistencyMetric

# Setting Up The Evaluator

Setting up the evaluator.

In [ ]:
from llama_index import (
    TreeIndex,
    VectorStoreIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    Response,
)
from llama_index.llms import OpenAI
from llama_index.evaluation import ResponseEvaluator

In [ ]:
import os
import openai

api_key = "sk-XXX"
openai.api_key = api_key

In [ ]:
gpt4 = OpenAI(temperature=0, model="gpt-4", api_key=api_key)
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)
evaluator_gpt4 = ResponseEvaluator(service_context=service_context_gpt4)

## Getting a LlamaHub Loader

In [ ]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()
documents = loader.load_data(pages=["Tokyo"])

In [ ]:
tree_index = TreeIndex.from_documents(documents=documents)
vector_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context_gpt4
)

We then build an evaluator based on the `BaseEvaluator` class that requires an `evaluate` method.

In this example, we show you how to write a factual consistency check.

In [ ]:
class FactualConsistencyResponseEvaluator:
    def get_context(self, response: Response) -> List[Document]:
        """Get context information from given Response object using source nodes.

        Args:
            response (Response): Response object from an index based on the query.

        Returns:
            List of Documents of source nodes information as context information.
        """
        context = []

        for context_info in response.source_nodes:
            context.append(Document(text=context_info.node.get_content()))

        return context

    def evaluate(self, response: Response) -> str:
        """Evaluate factual consistency metrics"""
        answer = str(response)
        context = self.get_context(response)
        metric = FactualConsistencyMetric()
        context = " ".join([d.text for d in context])
        score = metric.measure(output=answer, context=context)
        if metric.is_successful():
            return "YES"
        else:
            return "NO"


evaluator = FactualConsistencyResponseEvaluator()

In [ ]:
query_engine = tree_index.as_query_engine()
response = query_engine.query("How did Tokyo get its name?")
eval_result = evaluator.evaluate(response)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


{'success': True, 'score': 0.97732705}


/usr/local/lib/python3.10/dist-packages/deepeval/metrics/metric.py:42: UserWarning: API key is not set. Please set it by visiting https://app.confident-ai.com
  warnings.warn(


# Other Metrics

We recommend using other metrics to help give more confidence to various prompt iterations, LLM outputs etc. We think ML-assisted approaches are required to give performance for these models.

- Overall Score: https://docs.confident-ai.com/docs/measuring_llm_performance/overall_score
- Answer Relevancy: https://docs.confident-ai.com/docs/measuring_llm_performance/answer_relevancy
- Bias: https://docs.confident-ai.com/docs/measuring_llm_performance/debias